In [5]:
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from n2v.models import N2VConfig, N2V
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from csbdeep.utils import plot_history
from skimage import io
from skimage.transform import resize

MODEL_PATH = "output/models"
OUTPUT_PATH = "output/images"
def train_n2v_model_multi(train_images):
    """Train a Noise2Void model on multiple images"""
    datagen = N2V_DataGenerator()
    
    # Process each training image
    all_patches = []
    
    for img in train_images:
        # Prepare the image for N2V - add channel dimension
        img_for_patches = img[np.newaxis, ..., np.newaxis]
        
        # Generate patches for this image
        patch_shape = img_for_patches.shape[1:]
        patches = datagen.generate_patches_from_list([img_for_patches], shape=patch_shape)
        all_patches.append(patches)
    
    # Combine all patches
    X = np.concatenate(all_patches, axis=0)
    print(f"Generated a total of {X.shape[0]} training patches of shape {X.shape[1:]}")
    
    # Split into training and validation (80/20)
    np.random.shuffle(X)
    n_train = int(0.8 * X.shape[0])
    X_train, X_val = X[:n_train], X[n_train:]
    print(f"Training on {n_train} patches, validating on {X.shape[0] - n_train} patches")
    
    # Configure N2V
    config = N2VConfig(
        X_train,
        unet_kern_size=3,
        train_steps_per_epoch=max(int(X_train.shape[0]/128), 10), 
        train_epochs=15,  # 15 epochs as requested
        train_loss='mse',
        batch_norm=True,
        train_batch_size=128,
        n2v_perc_pix=0.198,
        n2v_patch_shape=(64, 64),
        n2v_manipulator='uniform_withCP',
        n2v_neighborhood_radius=5,
        # N2V2 improvements
        blurpool=True,
        skip_skipone=True,
        unet_residual=False
    )
    
    # Create and train the model
    model_name = 'n2v_cryoET_8slices'
    model = N2V(config, model_name, basedir=MODEL_PATH)
    
    print("Starting model training...")
    history = model.train(X_train, X_val)
    
    # Plot training history
    plt.figure(figsize=(10, 4))
    plot_history(history, ['loss', 'val_loss'])
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'training_history.png'))
    plt.show()
    
    return model

In [6]:
input_dir = "../parse_data/yolo_dataset/images/train"
img_extensions = ('.png', '.jpg', '.tif', '.tiff')
img_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.lower().endswith(img_extensions)]
TARGET_SHAPE = (64, 64)
# --- 画像の読み込みと [0,1] への正規化 ---
images = []
for file in img_files:
    img = io.imread(file)
    # グレースケール画像の場合、チャネル次元を追加
    if img.ndim == 2:
        img = img[..., np.newaxis]
    img = resize(
        img,
        (*TARGET_SHAPE, img.shape[-1]),
        preserve_range=True,
        anti_aliasing=True
    ).astype(np.float32)
    # 画像を [0,1] の範囲に正規化
    img = (img - img.min()) / (img.max() - img.min() + 1e-8)
    images.append(img)

# 画像データを numpy 配列にまとめる: shape -> (n, H, W, C)
images = np.array(images)

In [7]:
model = train_n2v_model_multi(images)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 2, the array at index 0 has size 64 and the array at index 1 has size 1

In [9]:
import os
import numpy as np
from skimage import io
from skimage.transform import resize
from n2v.models import N2V

# --- パスやパラメータの設定 ---
MODEL_PATH = 'output/denoise_model/'
input_dir = "../parse_data/yolo_dataset/images/train"

# 対象とする画像拡張子の定義
img_extensions = ('.png', '.jpg', '.tif', '.tiff')
img_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.lower().endswith(img_extensions)]
TARGET_SHAPE = (256, 256)
# --- 画像の読み込みと [0,1] への正規化 ---
images = []
for file in img_files:
    img = io.imread(file)
    # グレースケール画像の場合、チャネル次元を追加
    if img.ndim == 2:
        img = img[..., np.newaxis]
    img = resize(
        img,
        (*TARGET_SHAPE, img.shape[-1]),
        preserve_range=True,
        anti_aliasing=True
    ).astype(np.float32)
    # 画像を [0,1] の範囲に正規化
    img = (img - img.min()) / (img.max() - img.min() + 1e-8)
    images.append(img)

# 画像データを numpy 配列にまとめる: shape -> (n, H, W, C)
images = np.array(images)
print("Loaded images shape:", images.shape)

# --- 訓練用と検証用データへの分割 ---
split_ratio = 0.8  # 80% を訓練、20% を検証に使用
split_index = int(len(images) * split_ratio)
x_train = images[:split_index]
x_val = images[split_index:]
print("Training set shape:", x_train.shape)
print("Validation set shape:", x_val.shape)

# --- Noise2Void モデルの生成 ---
model = N2V(config=None, 
            #name='n2v_cryoET_8slices', 
            #basedir=MODEL_PATH
            )

# --- 学習パラメータの設定 ---
epochs = 200            # エポック数
steps_per_epoch = 200   # 1 エポックあたりのステップ数
batch_size = 4          # バッチサイズ

# --- モデルの学習 ---
model.train(x_train, x_val, steps_per_epoch=steps_per_epoch, epochs=epochs, batch_size=batch_size)

Loaded images shape: (3262, 256, 256, 1)
Training set shape: (2609, 256, 256, 1)
Validation set shape: (653, 256, 256, 1)


AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
# Batch processing function for denoising multiple slices
def batch_process_n2v(input_dir, output_dir, slice_range=None):
    """
    Process a batch of slices with the pre-trained Noise2Void model
    
    Parameters:
        input_dir: Directory containing input JPEG slices
        output_dir: Directory to save denoised slices
        slice_range: Optional tuple (start, end) to process only a subset of slices
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get all JPEG files in the input directory
    all_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')])
    
    # Apply slice range if specified
    if slice_range is not None:
        start, end = slice_range
        all_files = all_files[start:end]
    
    print(f"Processing {len(all_files)} slices...")
    
    # Process each slice
    for filename in all_files:
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, f"denoised_{filename}")
        
        # Load image
        img = np.array(Image.open(input_path).convert('L'))
        
        # Add dimensions for N2V (SYXC format)
        img_for_pred = img[np.newaxis, ..., np.newaxis]
        
        # Apply Noise2Void denoising
        denoised = model.predict(img_for_pred, axes='SYXC')
        
        # Remove batch and channel dimensions
        denoised_img = denoised[0, ..., 0]
        
        # Convert to uint8 for saving
        if denoised_img.dtype != np.uint8:
            denoised_img = np.clip(denoised_img, 0, 255).astype(np.uint8)
        
        # Save denoised image
        Image.fromarray(denoised_img).save(output_path)
    
    print(f"Batch processing complete. Denoised images saved to {output_dir}")

# tomo_id = '00e047'
# input_dir = f'/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/train/tomo_{tomo_id}'
# output_dir = f'/kaggle/working/denoised_tomo_{tomo_id}'
# batch_process_n2v(input_dir, output_dir, slice_range=(100, 200))

In [ ]:
input_dir = "../parse_data/yolo_dataset/images/train"
output_dir = "yolo_dataset/images/train"
batch_process_n2v(input_dir, output_dir)

In [ ]:
input_dir = "../parse_data/yolo_dataset/images/val"
output_dir = "yolo_dataset/images/val"
batch_process_n2v(input_dir, output_dir)